In [1]:
import json

from pathlib import Path

from smart_open import open
from gensim import corpora
from gensim.models import LdaModel
from gensim.test.utils import datapath

In [2]:
DATASET = "sample50k_health_tech"

PROCESSED_PATH = Path("../data/data_processed")
DICTIONARY_PATH = Path("../results/dictionary")
CORPUS_PATH = Path("../results/corpus")

### Create Gensim Dictionary

In [3]:
dictionary = corpora.Dictionary()

for fn in list((PROCESSED_PATH/DATASET).iterdir()):
    dictionary.add_documents(json.loads(line)["article"] for line in open(fn))

# remove words that appear only once
once_ids = [tokenid for tokenid, docfreq in dictionary.dfs.items() if docfreq == 1]
dictionary.filter_tokens(once_ids)  # remove words that appear only once
dictionary.compactify()  # remove gaps in id sequence after words that were removed

# save dictionary
dictionary.save((DICTIONARY_PATH/DATASET).as_posix())

### Vectorize corpus

Streaming tokenized corpus.

In [4]:
class MyCorpus(object):
    def __iter__(self):
        for fn in list((PROCESSED_PATH/DATASET).iterdir()):
            for line in open(fn):
                yield dictionary.doc2bow(json.loads(line)["article"])

corpus_memory_friendly = MyCorpus()
bow_corpus = [vector for vector in corpus_memory_friendly]

corpora.MmCorpus.serialize(f'{(CORPUS_PATH/DATASET).as_posix()}.mm', bow_corpus)